# Crystalization Notes

## 2017-10-09
Should have started this weeks ago.  Anyway, the idea is that we can use a variant of [this PPT by Cho](http://web.ecs.baylor.edu/faculty/cho/Cho-Entropy.pdf) to subgraphs of minimal energy.  

The ultimate goal is to take a patients CUI set and find the maximally fitting set of conditions for them.  The conditions should initially be informed by the "Conditions" listed with the CT.gov dataset, but at some point we'll let users construct their own.


### Recipe Reconstruction
The [Yummly Data](https://www.kaggle.com/c/whats-cooking) seems a nice analogy.  They provide a three-tiered data set that includes "Cuisine > Recipe > Ingredients".  [This guy](https://github.com/Btibert3/kaggle-yummly) tried it in Neo4j. The Recipe is just an idea. The intent of the competition was to be able to predict the cuisine given just the ingredients.  However, in our analogy I want to find the "maximally fitting" recipe for a given set of ingredients.  We don't have labels for this, so I don't have the success metric yet.

I took a set of about 124 recipes to run the minimal energy routine on.  Unfortunately, I messed up the code and I mistook it for not doing the minimization.  Specifically, I we returning the size of `cupboard` which is just the set of ingredient ids instead of returning the output of `minEnergy`.  To solve the problem I didn't have, I created the `knockout` routine, which removes a few ingredients from each recipe before it gets sent in for minimization.

The `knockout` routine may or may not be causing very small crystals.  I'm running some metrics on that now.  However, `knockout` isn't necessary right now since the error never really existed.

I think one of the issues it that the crystals are small because the data set is not very connected. In the 124 set, we don't get much coverage of any one cuisine, and though I'm not sure how to test it, I'm thinking highly specific ingredients are experiencing very low degrees.

I still feel confident the method will be informative, but it's not being as cooperative as I'd like.

Today, I think I'm going to try to extract all "Italian" recipes or something like that.  Actually, I should probably pull the whole thing into Postgres so I can sub-set as needed.

```
DROP TABLE IF EXISTS r.yummly;
CREATE TABLE r.yummly
(cuisine text
, recipe int
, ingredient text
);
\copy r.yummly from yummly.txt 
```

### SQL EDA

```
select cuisine, count(distinct(recipe)), count(distinct(ingredient)) 
from r.yummly 
group by 1 
order by 1;

   cuisine    | count | count
--------------+-------+-------
 brazilian    |   467 |   853
 british      |   804 |  1166
 cajun_creole |  1546 |  1576
 chinese      |  2673 |  1792
 filipino     |   755 |   947
 french       |  2646 |  2102
 greek        |  1175 |  1198
 indian       |  3003 |  1664
 irish        |   667 |   999
 italian      |  7838 |  2929
 jamaican     |   526 |   877
 japanese     |  1423 |  1439
 korean       |   830 |   898
 mexican      |  6438 |  2684
 moroccan     |   821 |   974
 russian      |   489 |   872
 southern_us  |  4320 |  2462
 spanish      |   989 |  1263
 thai         |  1539 |  1376
 vietnamese   |   825 |  1108
```